In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dhritiwasan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dhritiwasan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import os
import gensim

# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  

stop_words = set(stopwords.words('english')) 

In [3]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
ps = PorterStemmer()
lemma = WordNetLemmatizer()

In [4]:
words = set()

for f in sorted(os.listdir('songs')):
    if f[0] == '.':
        continue
    
    for i, l in enumerate(open('songs/' + f, 'r').readlines()):
        
        if i < 2:
            continue
        
        arr = word_tokenize(l)
        for a in arr:
            a = lemma.lemmatize(a.lower())
            if "'" in a:
                continue
            if len(a) < 3:
                continue
            if not a in model.vocab:
                continue
            if a in stop_words:
                continue
            words.add(a)

print(words)

{'grow', 'move', 'winding', 'loved', 'million', 'flower', 'bloom', 'working', 'truly', 'gem', 'marching', 'said', 'still', 'guiding', 'flow', 'home', 'get', 'vision', 'twinkling', 'dot', 'moment', 'build', 'fall', 'sunrise', 'striving', 'mind', 'echoing', 'united', 'bright', 'thus', 'soul', 'fly', 'feeling', 'exicted', 'heat', 'starting', 'passed', 'different', 'thousand', 'youll', 'seems', 'unfold', 'long', 'pretty', 'want', 'think', 'one', 'toil', 'add', 'dark', 'white', 'reaching', 'catch', 'road', 'conviction', 'star', 'rainbow', 'strive', 'recall', 'yearning', 'especially', 'prepared', 'darkest', 'bus', 'hollywood', 'air', 'surprise', 'grown', 'riverside', 'take', 'sweet', 'democratic', 'turning', 'built', 'beginning', 'let', 'join', 'knew', 'hand', 'carried', 'refreshed', 'near', 'greenery', 'stranger', 'open', 'surely', 'low', 'come', 'live', 'core', 'high', 'overcome', 'francisco', 'quay', 'spot', 'history', 'sound', 'tame', 'world', 'truth', 'belongs', 'beautiful', 'beat', 'fl

In [5]:
import numpy as np
import copy

centroids = np.zeros((len(words), 300))

for i, w in enumerate(words):
    centroids[i] = model[w]
    

In [6]:
dist = np.zeros((len(words), len(words)))
for i in range(dist.shape[0]):
    for j in range(dist.shape[1]):
        dist[i, j] = np.linalg.norm(centroids[i] - centroids[j])
out = open('dist.txt', 'w')
for i in range(dist.shape[0]):
    for j in range(dist.shape[1]):
        out.write(str(dist[i, j]).ljust(20) + ' ')
    out.write('\n')
out.close()

In [7]:
dist.shape

(544, 544)

In [8]:
import matplotlib.pyplot as plt

plt.plot(centroids[:,2], centroids[:,3], 'o')
plt.show()

<Figure size 640x480 with 1 Axes>

In [9]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components=50).fit(centroids)
labels = gmm.predict(centroids)
mapping = {}
for s, l in sorted(zip(words, labels), key = lambda x: x[1]):
    mapping[s] = l

In [10]:
songs = sorted(os.listdir('songs'))
mfw = np.zeros((len(songs), 50))

for index, f in enumerate(songs):
    if f[0] == '.':
        continue
    
    for i, l in enumerate(open('songs/' + f, 'r').readlines()):
        
        if i < 2:
            continue
        
        arr = word_tokenize(l)
        for a in arr:
            a = lemma.lemmatize(a.lower())
            if "'" in a:
                continue
            if len(a) < 3:
                continue
            if not a in model.vocab:
                continue
            if a in stop_words:
                continue
            mfw[index, mapping[a]] = 1
            
print(mfw)

[[0. 1. 0. ... 1. 0. 1.]
 [0. 1. 1. ... 0. 0. 1.]
 [0. 1. 1. ... 0. 0. 1.]
 ...
 [0. 1. 1. ... 0. 1. 1.]
 [0. 1. 1. ... 0. 0. 1.]
 [0. 1. 1. ... 0. 0. 1.]]


In [11]:
dist = np.zeros((len(songs), len(songs)))
for i in range(dist.shape[0]):
    for j in range(dist.shape[1]):
        dist[i, j] = np.linalg.norm(mfw[i] - mfw[j])

In [12]:
dist.shape

(29, 29)

In [17]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components=5).fit(mfw)
labels = gmm.predict(mfw)
for s, l in sorted(zip(songs, labels), key = lambda x: x[1]):
    print(s.ljust(8),l)

2007.txt 0
1962.txt 1
1967.txt 1
1969.txt 1
1999.txt 1
2000.txt 1
2001.txt 1
2002.txt 1
2004.txt 1
2006.txt 1
2007-2.txt 1
2012.txt 1
2015.txt 1
1984.txt 2
1986.txt 2
1990.txt 2
1991.txt 2
2003.txt 2
2009.txt 2
2013.txt 2
2017.txt 2
1997.txt 3
1999-2.txt 3
2005.txt 3
2008.txt 3
2010.txt 3
2011.txt 3
2016.txt 3
1987.txt 4


In [14]:
import sys
numpy.set_printoptions(threshold=sys.maxsize)
print(mfw)

NameError: name 'numpy' is not defined